In [1]:
import openai
import json

def parent_annotate(folder):
    #the full folder has a list of folders with a name
    #for each folder in the parent folder
    #we need to open the "task" folder
    #open the brief.md file
    #send in as input the entire brief.md file as the user prompt to gpt and gemini
    #set the system prompt manually for both models based on the final decided prompt
    #store the output nested list into a json file 
    #keys will be folder name 
    #values will be gpt output and gemini output
    final_json = []
    return final_json

In [2]:
sys_prompt = """


You are a task classifier model. You will be provided with a task in the form of a brief. This brief will contain deliverables, what needs to be done, and what is provided. In the real world - this brief was provided to an actual human who completed the task. We are a team of researchers interested in knowing how this task should be classified. Below you will find a task classification taxonomy. 
Please read about the skills carefully and take note of the various subskills that might be part of each skill. Look at the examples. 

Capabilities - 4
Skills - 13

Capability 1 - Technical Capabilities
Skills - 

SKILL NAME - Data and Information Processing -  
    
    Example Sub-Skill - Data Collection 
    Definions - Gathering and extracting information from various digital sources
    Examples - Setting up web scraping pipelines to gather competitive intelligence
    O*Net Reference - O*NET Basic Skills: Active Learning, Reading Comprehension

    Example Sub-Skill - Data Organization
    Definions - Structuring and categorizing information in digital systems
    Examples - Creating a taxonomy system for an enterprise content management system
    O*Net Reference - O*NET Cross-Functional Skills: Management of Information Resources

    Example Sub-Skill - Data Analysis
    Definions - Interpreting and deriving insights from digital information
    Examples - Building dashboards that identify sales trends across product categories
    O*Net Reference - O*NET Cross-Functional Skills: Systems Analysis, Mathematics

    Example Sub-Skill - Information Evaluation
    Definions -Assessing the quality, reliability and relevance of digital content
    Examples - Verifying credibility of sources for research and identifying misinformation
    O*Net Reference - O*NET Basic Skills: Critical Thinking


SKILL NAME - Digital Tool Operations - Effectively operating specialized digital applications 

    
    Example Sub-Skill - Software Navigation
    Definions -Effectively operating specialized digital applications
    Examples - Using advanced features in Adobe Creative Suite with efficient workflows
    O*Net Reference - O*NET Technical Skills: Operation and Control

    Example Sub-Skill - Hardware Management
    Definions -Operating and maintaining physical computing devices
    Examples - Configuring and troubleshooting multi-monitor workstations and peripherals
    O*Net Reference - O*NET Technical Skills: Equipment Maintenance

    Example Sub-Skill - Platform Utilization
    Definions -Working effectively within specific digital environments
    Examples - Managing an enterprise CMS while customizing workflows for different teams
    O*Net Reference - O*NET Knowledge: Computers and Electronics

    Example Sub-Skill - Technical Troubleshooting
    Definions -Identifying and resolving digital system issues
    Examples - Diagnosing and fixing software conflicts between integrated applications
    O*Net Reference - O*NET Technical Skills: Troubleshooting, Equipment Maintenance



SKILL NAME - Digital Content Creation 
    
    Example Sub-Skill - Text Production
    Definions -Creating written digital content for various purposes
    Examples - Writing SEO-optimized blog content that drives engagement and conversion
    O*Net Reference - O*NET Basic Skills: Writing

    Example Sub-Skill - Visual Development
    Definions -Producing digital imagery and design elements
    Examples - Creating data visualizations that communicate complex insights effectively
    O*Net Reference - O*NET Cross-Functional Skills: Technology Design

    Example Sub-Skill - Multimedia Production
    Definions -Creating audio and video content
    Examples - Producing and editing professional-quality video tutorials with animations
    O*Net Reference - O*NET Cross-Functional Skills: Technology Design

    Example Sub-Skill - Interactive Content Development
    Definions -Building responsive user experiences
    Examples - Developing interactive assessments with branching scenarios based on user input
    O*Net Reference - O*NET Cross-Functional Skills: Programming



SKILL NAME - Programming and Development

    Example Sub-Skill - Code Writing
    Definions -Creating functional programming scripts and applications
    Examples - Developing Python scripts to automate data processing workflows
    O*Net Reference - O*NET Cross-Functional Skills: Programming

    Example Sub-Skill - Application Development
    Definions -Building comprehensive software applications
    Examples - Creating a mobile app that integrates with enterprise systems
    O*Net Reference - O*NET Cross-Functional Skills: Programming, Technology Design

    Example Sub-Skill - System Integration
    Definions -Connecting different digital platforms and systems
    Examples - Implementing API connections between CRM and marketing automation systems
    O*Net Reference - O*NET Cross-Functional Skills: Systems Evaluation

    Example Sub-Skill - Technology Implementation
    Definions -Deploying and configuring digital solutions
    Examples - Setting up cloud infrastructure with appropriate security configurations
    O*Net Reference - O*NET Cross-Functional Skills: Installation



Capability 2 - Cognitive Skills 
Skills - 

SKILL - Analytical Thinking
    Example Sub-Skill - Pattern Recognition
    Definions -Identifying meaningful trends and relationships in data
    Examples - Detecting user behavior patterns in web analytics to optimize conversion
    O*Net Reference - O*NET Abilities: Inductive Reasoning

    Example Sub-Skill - Logical Reasoning
    Definions -Applying structured thinking to digital problems
    Examples - Developing decision trees for automated customer support routing
    O*Net Reference - O*NET Abilities: Deductive Reasoning

    Example Sub-Skill - Comparative Analysis
    Definions -Evaluating alternatives based on specific criteria
    Examples - Assessing competing technology solutions against business requirements
    O*Net Reference - O*NET Cross-Functional Skills: Judgment and Decision Making

    Example Sub-Skill - Systems Thinking
    Definions -Understanding interconnected digital components
    Examples - Mapping dependencies in microservice architectures to plan scaling
    O*Net Reference - O*NET Cross-Functional Skills: Systems Evaluation, Systems Analysis

SKILL - Creative Thinking
    Example Sub-Skill - Idea Generation
    Definions -Producing novel concepts and approaches
    Examples - Brainstorming innovative UX features for digital products
    O*Net Reference - O*NET Abilities: Fluency of Ideas, Originality

    Example Sub-Skill - Innovative Problem Solving
    Definions -Developing unconventional solutions to challenges
    Examples - Creating workarounds for API limitations in legacy systems
    O*Net Reference - O*NET Cross-Functional Skills: Complex Problem Solving

    Example Sub-Skill - Conceptual Design
    Definions -Deploying and configuring digital solutions
    Examples - Developing wireframes for novel augmented reality experiences
    O*Net Reference - O*NET Abilities: Visualization

    Example Sub-Skill - Creative Adaptation
    Definions -Modifying existing approaches for new contexts
    Examples - Repurposing e-commerce strategies for educational 
    O*Net Reference - O*NET Cross-Functional Skills: 

SKILL - Critical Thinking 
    Example Sub-Skill - Information Verification
    Definions -Validating accuracy and reliability of digital information
    Examples - Fact-checking claims in research reports against primary sources
    O*Net Reference - O*NET Basic Skills: Critical Thinking

    Example Sub-Skill - Assumption Identification
    Definions -Recognizing underlying premises in digital contexts
    Examples - Identifying hidden biases in algorithmic decision systems
    O*Net Reference - O*NET Basic Skills: Critical Thinking

    Example Sub-Skill - Perspective Analysis
    Definions -Considering multiple viewpoints on digital issues
    Examples - Evaluating digital products from different cultural perspectives
    O*Net Reference - O*NET Social Skills: Social Perceptiveness

    Example Sub-Skill - Evidence Evaluation
    Definions -Assessing the quality of digital information sources
    Examples - Determining confidence levels in predictive analytics models
    O*Net Reference - O*NET Cross-Functional Skills: Judgment and Decision Making


Capability 3 - Digital Communication
Skills - clear message writing 
    Definiton - Creating clear written messages in digital formats 
    Example - Crafting concise and action-oriented emails that achieve objectives
    O*Net Reference - O*NET Basic Skills: Writing

Skills - visual aid usage -
    Definition - Using images and visual elements to convey information
    Example - Creating infographics that simplify complex technical concepts
    O*Net Reference - O*NET Cross-Functional Skills: Technology Design


Skills - Multimedia -
    Definition - Using audio and video for effective messaging
    Example - Producing engaging video presentations for remote stakeholders
    O*Net Reference- O*NET Basic Skills: Speaking

Skills - audience calibration -
    Definition - Tailoring digital communication to specific recipients
    Example - Customizing technical documentation for different expertise levels
    O*Net Reference - O*NET Social Skills: Social Perceptiveness

Capability 4 = Domain Knowledge

SKILL - Industry Specific Knowledge 

    Example Sub-Skill - Vertical Industry Understanding
    Definions -Sector-specific digital needs and applications
    Examples - Understanding healthcare data privacy requirements for medical systems
    O*Net Reference - O*NET Knowledge: Industry-specific knowledge domains

    Example Sub-Skill - Specialized Platforms
    Definions -Industry-specific digital systems and tools
    Examples - Operating electronic health record systems in clinical settings
    O*Net Reference - O*NET Knowledge: Computers and Electronics

    Example Sub-Skill - Sector Terminology
    Definions -Industry-specific digital language and concepts
    Examples - Understanding cybersecurity threat classification terminology
    O*Net Reference -O*NET Knowledge: Domain-specific terminology

    Example Sub-Skill - Industry Digital Trends
    Definions -Emerging developments in sector-specific technology
    Examples - Following emerging fintech innovations for financial services
    O*Net Reference - O*NET Work Activities: Keeping Up to Date Technically

SKILL - Functional Area Knowledge

    Example Sub-Skill - Technical Domain Expertise
    Definions - Specialized technical knowledge
    Examples - Deep understanding of database optimization techniques
    O*Net Reference - O*NET Knowledge: Computers and Electronics

    Example Sub-Skill - Business Function Knowledge
    Definions - Operational area understanding
    Examples - Knowledge of digital marketing principles and metrics
    O*Net Reference - O*NET Knowledge: Sales and Marketing

    Example Sub-Skill - Process Expertise
    Definions -Workflow and methodology knowledge
    Examples - Understanding agile development methodologies
    O*Net Reference -O*NET Knowledge: Administration and Management

    Example Sub-Skill - Tool Specialization
    Definions -Expertise with specific applications
    Examples - Expert-level knowledge of business intelligence platforms
    O*Net Reference - O*NET Cross-Functional Skills: Operation and Control


You will be given a brief - then you will read the entire brief and return four python lists. Each one-hot encoded for the skills you think this job needs. So if job needs digital tool operation, programming and development, creative thinking, visual aid usage, audience calibration, functional area knowledge, you will return 

[[0,1,0,1], [0,1,0], [0,1,0,1], [0,1]]

Please only return the vector and nothing else. Do not start your sentences or outputs with "sure here's ..." simply return the vector and stop outputs. 





"""

In [3]:
from dotenv import load_dotenv
import os
from litellm import completion
import google.generativeai as genai
import os
from openai import OpenAI


load_dotenv()  # Load from .env file in the current dir

api_key = os.getenv("litellm_API_KEY")
#print(api_key)




BASE_URL = "https://litellm.ml-serving-internal.scale.com/v1"
api_key = api_key

client = OpenAI(
    api_key=api_key,
    base_url=BASE_URL,
)


INTERNAL_USER = "internal"
cost_tags = {
    "metadata": {
        "tags": ["project:agent-arena"]
    }
}


response = client.chat.completions.create(
    model="gemini/gemini-2.0-flash",
    messages=[
        {"role": "system", "content": "say hello"},
        {"role": "user", "content":"say what model you are"}
    ],
    user=INTERNAL_USER,
    extra_body=cost_tags,
)

print(response.choices[0].message.content)
#sanity check - should say trained by google

I am a large language model, trained by Google.



In [5]:
import os
import json
import glob
from docx import Document
import openai
import google.generativeai as genai

# Define or import your system prompt, client, user, cost_tags as needed
# Example placeholder:




def parent_annotate(parent_folder_path: str) -> dict:
    """Iterate over sub-directories inside *parent_folder_path*, read any
    `brief.md`, `brief.txt`, or `brief.docx` file in each `task/` folder,
    and annotate it using GPT-4o and Gemini. Returns a dictionary keyed by
    sub-folder name with both model outputs."""

    results: dict[str, dict[str, str]] = {}

    if not os.path.isdir(parent_folder_path):
        raise FileNotFoundError(f"Parent folder not found: {parent_folder_path}")

    for folder in filter(lambda p: os.path.isdir(os.path.join(parent_folder_path, p)),
                         os.listdir(parent_folder_path)):
        task_path = os.path.join(parent_folder_path, folder, "task")
        brief_files = glob.glob(os.path.join(task_path, "brief.*"))
        brief = ""

        for file_path in brief_files:
            ext = os.path.splitext(file_path)[1].lower()
            try:
                if ext in [".md", ".txt"]:
                    with open(file_path, "r", encoding="utf-8") as f:
                        brief = f.read()
                        print("here")
                    break
                elif ext == ".docx":
                    doc = Document(file_path)
                    brief = "\n".join(p.text for p in doc.paragraphs)
                    print("docx here")
                    break
            except Exception:
                continue

        if not brief:
            brief = "No brief found - if you encounter this prompt simply enter a list of all zeroes len 13"

        # GPT‑4o
        try:
            gpt_resp = client.chat.completions.create(
                model="openai/o3",
                messages=[
                    {"role": "system", "content": sys_prompt},
                    {"role": "user", "content": brief}
                ],
                user=INTERNAL_USER,
                extra_body=cost_tags,
            )
            gpt_output = gpt_resp.choices[0].message.content.strip()
        except Exception as e:
            gpt_output = "nothing"

        # Gemini
        try:
            gemini_resp = client.chat.completions.create(
                model="gemini/gemini-2.0-flash",
                messages=[
                    {"role": "system", "content": sys_prompt},
                    {"role": "user", "content": brief}
                ],
                user=INTERNAL_USER,
                extra_body=cost_tags,
            )
            gemini_output = gemini_resp.choices[0].message.content.strip()
        except Exception as e:
            gemini_output = "nothing"

        results[folder] = {
            "gpt_output": gpt_output,
            "gemini_output": gemini_output,
        }

    with open("annotation_results_new_sys_prompt.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)

    return results


if __name__ == "__main__":
    parent_dir = "/Users/ankit.aich/Documents/Code/accepted_data"
    output = parent_annotate(parent_dir)
    print(output)


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
docx here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
docx here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
